# Numerical data in `numpy`

`numpy` is a very powerful library for working with numerical data in Python. It introduces the `array` data structure, which can contain multi-dimensional numerical data.

The `numpy` library is __not__ part of the Python standard library. However, it comes bundles with [Anaconda](01_anaconda.ipynb) so you should already have it installed. The usual way to import `numpy` is as follows:

    import numpy as np
    
This gives us access to all the `numpy`-functions using the prefix `np`. This is a convention, and you should do the same in your own code.

In [1]:
import numpy
numpy.__version__

'1.11.3'

## Reading data with `numpy`

The `numpy` library comes with a few functions that can read numerical data from text files. The `np.loadtxt`-function is a fast reader with some basic functionality for skipping header lines, converting values to floats, etc. For more sophisticated files the `np.genfromtxt`-function can be used. It also supports handling missing values.

Note that these functions take a filename as input, so that you do not need to use the built-in `open`-function to open a file beforehand.